In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name = "gpt-4o-mini",
    temperature=0.1
)

### from_template() 메소드
* 변수를 중괄호로 묶어서 템플릿에 정의

In [3]:
from langchain_core.prompts import PromptTemplate

# template 정의, {}안의 내용은 이후에 값이 들어갈 자리
template = "{name}의 직업은 무엇인가요?"

# from_template 메소드를 사용해 PromptTemplate 객체를 생성
prompt = PromptTemplate.from_template(template)

print(prompt)

input_variables=['name'] input_types={} partial_variables={} template='{name}의 직업은 무엇인가요?'


In [4]:
# prompt 생성 (완성) format() 메소드를 이용해 변수에 값을 넣음
prompt = prompt.format(name="bear")

print(prompt)

bear의 직업은 무엇인가요?


In [11]:

from langchain_core.prompts import PromptTemplate

template = "{langauge}는 누가 만들었나요?"

prompt = PromptTemplate.from_template(template)

print(prompt)

chain = prompt | llm

input_variables=['langauge'] input_types={} partial_variables={} template='{langauge}는 누가 만들었나요?'


In [9]:
# 기본적으로는 딕셔너리 형태로 변수를 작성해야 하지만
# 변수가 한개일때는 변수만으로 작성이 가능

# print(chain.invoke({"langauge" : "Python"}).content)
print(chain.invoke({"Python"}).content)

Python은 귀도 반 로섬(Guido van Rossum)에 의해 1991년에 처음 개발되었습니다. 그는 이 프로그래밍 언어를 만들 때 코드의 가독성과 사용의 용이성을 중시하였으며, Python은 이후로도 많은 개발자들 사이에서 인기를 끌고 있습니다.


### PromptTemplate 객체 생성과 도잇에 prompt 생성
* input_variables 인자를 사용해 변수를 지정한다.
* 템플릿에 작성한 변수가 input_variables에 없으면 예외를 발생시켜준다.

In [12]:
# template 정의
template = "{language}는 누가 만들었나요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["language"],
    
)

print(prompt)

prompt.format(language="Python")

input_variables=['language'] input_types={} partial_variables={} template='{language}는 누가 만들었나요?'


'Python는 누가 만들었나요?'

**partial_variables**
* 연산중 미리 계산된 변수를 프롬프트 템플릿에 지정해 넣을 수 있다.
* 항상 공통된 방식으로 가져오고 싶은 변수가 있는 경우 사용<br>
ex) 날짜, 시간

In [13]:
# template 정의
template = "{language1}과 {language2}는 각각 누가 만들었나요?"

# PromptTemplate 객체 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["language1"],
    partial_variables={
        "language2" : "Java" # dictionary 형태로 partial_variables 작성
    }
)

print(prompt)

input_variables=['language1'] input_types={} partial_variables={'language2': 'Java'} template='{language1}과 {language2}는 각각 누가 만들었나요?'


In [15]:
prompt.format(language1="Python")

'Python과 Java는 각각 누가 만들었나요?'

**partial()**
* 연산중에 미리 계산된 변수를 픔롬프트 템플릿에 지정해서 넣을 수 있다.

In [16]:
prompt_partial = prompt.partial(language2="JavaScript")

print(prompt_partial)

input_variables=['language1'] input_types={} partial_variables={'language2': 'JavaScript'} template='{language1}과 {language2}는 각각 누가 만들었나요?'


In [17]:
prompt_partial.invoke("Python")

StringPromptValue(text='Python과 JavaScript는 각각 누가 만들었나요?')

In [18]:
chain = prompt_partial | llm

print(chain.invoke("Python").content)

Python은 귀도 반 로썸(Guido van Rossum)에 의해 1991년에 처음 개발되었습니다. 그는 Python을 만들 때 코드의 가독성을 중요시하며, 프로그래밍 언어의 사용을 쉽게 하려는 목표를 가지고 있었습니다.

JavaScript는 브렌던 아이크(Brendan Eich)에 의해 1995년에 개발되었습니다. 그는 당시 넷스케이프(Netscape)에서 일하고 있었으며, 웹 브라우저에서 동적인 기능을 추가하기 위해 JavaScript를 만들었습니다. JavaScript는 처음에는 "Mocha"라는 이름으로 불렸고, 이후 "LiveScript"를 거쳐 현재의 이름으로 변경되었습니다.


### 파일로 부터 template 읽어오기
* prompt를 편하게 작성 및 수정
* 상황에 맞게 파일을 작성해두면 필요할때바다 꺼내서 쓸수있다.

In [19]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/language_simple.yaml", encoding="utf-8")

print(prompt)

input_variables=['language'] input_types={} partial_variables={} template='{language} 언어에 대해 3줄로 설명해줘.'


In [20]:
prompt.format(language="Python")

'Python 언어에 대해 3줄로 설명해줘.'

In [21]:
prompt2 = load_prompt("prompts/language.yaml", encoding="utf-8")

print(prompt2.format(language="Python"))

Python 언어에 대해 설명해주세요.
언어의 특징을 다음의 양식에 맞게 정리하세요
300자 내외로 작성하세요.
한글로 작성하세요.
---
#양식
1. 특징
2. 제작자
3. 대표적인 프레임워크
4. 많이 사용되는 분야



In [23]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt2 | ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0) | StrOutputParser()

answer = chain.invoke("Java")

print(answer)

1. 특징: Java는 객체 지향 프로그래밍 언어로, 플랫폼 독립성을 강조합니다. "Write Once, Run Anywhere"라는 슬로건 아래, JVM(Java Virtual Machine)을 통해 다양한 운영체제에서 실행 가능합니다. 강력한 메모리 관리와 자동 가비지 컬렉션 기능을 제공하며, 풍부한 라이브러리와 API를 지원합니다.

2. 제작자: Java는 1995년 썬 마이크로시스템즈(Sun Microsystems)의 제임스 고슬링(James Gosling)에 의해 개발되었습니다.

3. 대표적인 프레임워크: Spring, Hibernate, JavaServer Faces(JSF), Apache Struts 등이 있습니다.

4. 많이 사용되는 분야: Java는 웹 애플리케이션, 모바일 애플리케이션(특히 안드로이드), 기업용 소프트웨어, 빅데이터 처리 및 클라우드 컴퓨팅 등 다양한 분야에서 널리 사용됩니다.


### ChatPrompTemplate
* 대화 목록을 프롬프트로 주입하고자 할 때 사용 할 수 있다.
* 메시지는 튜플형태로 전달
    * ("role", "message") 구성되고, 리스트 형태로 생성 가능

**role**
* system : 시스템 설정메시지로 주로 전역서정을 할때 사용
* human : 사용자의 입력 메시지
* ai : AI의 답변 메시지

In [27]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{language}의 제작자는 누구인가요?")

print(chat_prompt)

input_variables=['language'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='{language}의 제작자는 누구인가요?'), additional_kwargs={})]


In [29]:
print(chat_prompt.format(language="python"))

Human: python의 제작자는 누구인가요?


In [30]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 인공지능 어시스턴트 입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}")
    ]
)

messages = chat_template.format_messages(
    name="무찌", user_input="당신의 이름은 무엇입니다?"
)

print(messages)

[SystemMessage(content='당신은 친절한 인공지능 어시스턴트 입니다. 당신의 이름은 무찌 입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='당신의 이름은 무엇입니다?', additional_kwargs={}, response_metadata={})]


In [31]:
print(llm.invoke(messages).content)

제 이름은 무찌입니다! 당신과 대화하게 되어 기쁩니다. 어떤 이야기를 나눌까요?


### MessagePlaceHolder
* 아직 확정된 메시지가 아니지만, 나중에 채워질 메시지 위치를 잡아두기 위해 사용한다.
* 보통 대화 기록을 하고싶을때 사용한다.

In [32]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것 입니다."
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다.")
    ]
)

print(chat_prompt)

input_variables=['conversation', 'word_count'] input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.m

In [34]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?"),
        ("ai", "아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다."),
        ("human", "set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ"),
        ("ai", "네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊")
    ]
)

print(formatted_chat_prompt)

System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것 입니다.
Human: 파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?
AI: 아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다.
Human: set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ
AI: 네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊
Human: 지금까지의 대화를 5 단어로 요약합니다.


In [35]:
chain = chat_prompt | llm | StrOutputParser()

In [36]:
input = {
    "word_count":5,
    "conversation":[
        ("human", "파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?"),
        ("ai", "아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다."),
        ("human", "set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ"),
        ("ai", "네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊")
    ]
}

chain.invoke(input)

'파이썬, 리스트, 중복 제거, set.'

In [ ]:
!pip install langchain-community

In [39]:
from langchain_community.llms import Ollama

# model
llm = Ollama(model="gemma2:9b")

response = llm.invoke("지구의 자전 주기는?")

response

' 지구의 자전 주기는 약 24시간입니다. 즉, 지구가 한 바퀴 돌아서 하늘을 다시 본다는 것을 의미합니다. 이 기간은 "하루"라고 불립니다.\n\n\n하지만 실제로 지구의 자전 주기는 정확히 23시간 56분 4초라는 더 정교한 값입니다. 이 차이는 태양광이 지구를 감쌀 때까지 지구가 회전하는 시간과 태양을 관찰할 때 나타나는 시간 사이의 차이 때문입니다.\n\n* **시차**: 각 지역은 해당 기준 시각에 따른 본래 자전 주기를 따라서 24시간으로 설정된 하루를 사용합니다. 그러나 진정한 지구의 자전 주기는 태양과의 상호 작용을 통해 달라질 수 있습니다.\n\n\n'

In [40]:
from langchain.chat_models import ChatOllama
model = ChatOllama(
    model = "gemma2:9b",
    temperature=0.1
)


C:\Users\20118\AppData\Local\Temp\ipykernel_17464\2870157151.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(


In [41]:
chain = chat_prompt | model | StrOutputParser()

chain.invoke(input)

'파이썬, 중복 제거, set 사용, 간단 해결, 유용  \n'